# PINECONE

# Load Dependencies 👩‍💻👨‍💻

Antes de ejecutar:
- Cargar una carpeta docs/ con archivos .txt que serán la knowledge base
- Usar T4 en el ambiente o modificar 'cuda' a 'gpu'
- Cargar un archivo .env con las API KEYS de HuggingFace/OpenAI y Pinecone

In [ ]:
pip install openai langchain langchain-community pinecone-client python-dotenv langchain-pinecone tiktoken protoc_gen_openapiv2 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langchain-core
    Fo

In [ ]:
import openai
import langchain
import os
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import logging

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

logging.basicConfig(level=logging.DEBUG,
                    format='[%(levelname)s] - %(message)s ',
                    handlers=[
                        logging.FileHandler('/content/langchaindemo.log', mode='w'),
                        logging.StreamHandler(),
                    ],
                    force=True)
logger = logging.getLogger(__name__)
logger.info("Langchain Demo Initialized")
logging.getLogger("openai").setLevel(logging.WARNING)

[INFO] - Langchain Demo Initialized 


# Document Loader 📑📚

Load data from a source as Document's. A Document is a piece of text and associated metadata.

https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html

In [ ]:
def get_docs():
    """
    Loads each file into one document (knowledge base)
    :return: docs
    """

    loader = DirectoryLoader(  # Reads custom data from local files
        path="docs",
        glob="*.txt",
        loader_cls=TextLoader  # Loader class to use for loading files
    )

    docs = loader.load()
    return docs

Create a Document class with metadata and content

In [ ]:
docs = get_docs()
docs[1]

[DEBUG] - Processing file: docs/temario.txt 
[DEBUG] - Processing file: docs/materia.txt 
[DEBUG] - Processing file: docs/alumnos.txt 


Document(metadata={'source': 'docs/materia.txt'}, page_content='Denominación de la materia: Temas Avanzados de Deep Learning\n\n\n\nDocente responsable de la materia:  Eugenia Piñeiro\n\nEquipo docente: Marina Fuster y Eugenia Piñeiro\n\nCarga horaria total: 24 horas\n\nPresentación de la materia:\nEsta materia se enfoca en dos aspectos cruciales de la temática modelos de lenguaje. Por un lado, busca profundizar en el aspecto técnico de los mismos, construyendo sobre los pasos anteriores que llevan hasta su reciente desarrollo. \nPor otro lado, se tiene como objetivo entender el estado del arte y los desafíos teórico-prácticos que se encuentran abiertos en la actualidad. \n\nObjetivos de aprendizaje:\nExponer a los alumnos a las generalidades del funcionamiento de la red neuronal Transformer.\nEntender las particularidades y desafíos de los distintos procesos involucrados en el armado y el entrenamiento de un modelo de lenguaje.\nEntender las diferentes áreas en las cuales se basa el c

In [ ]:
docs[0]

Document(metadata={'source': 'docs/temario.txt'}, page_content='Contenidos:\nUnidad 1: Introducción a Transformers\nIntroducción al concepto de Gen AI, LLMs y Transformers. Historia. Arquitectura. Mecanismo de Atención. Embeddings y Positional Encoding. Aplicaciones en la industria.\n\nUnidad 2: Algoritmos de Embedding y Positional Encoding\nAlgoritmos de Embedding y Positional Encoding. Transformer basando en N-grama\n\nUnidad 3: Fine Tuning\nReinforcement Learning. RLHF y sus security issues. Fine tuning. Pipeline productivo. \n\nUnidad 4: Responsible AI\nConsideraciones éticas en AI: biases en training data, fairness, impacto social, detección de contenido generado de forma artificial. Narrow AI vs. AGI. AGI como agente. Foundation models. Emergent capabilities. Security vulnerabilities. Interpretability. Alignment.\n\nUnidad 5: Retrieval Augmented Generation (RAG)\nIntroducción Retrieval Augmented Generation. Bases de datos vectoriales: Chroma DB y Pinecone, uso de embeddings y efi

# Text Splitters 📖✂️

Split a long document into smaller chunks that can fit into your model's context window

https://js.langchain.com/v0.1/docs/modules/data_connection/document_transformers/

Los separadores pueden ser pasados como argumento y se puede usar la funcion 'create_documents' si no utilizamos un DocumentLoader

In [ ]:
def get_chunks(docs, chunk_size=1000, chunk_overlap=200):
    """
    Get chunks from docs. Our loaded doc may be too long for most models, and even if it fits it can struggle to find relevant context. So we generate chunks
    :param docs: docs to be splitted

    :return: chunks
    """

    text_splitter = RecursiveCharacterTextSplitter( # recommended splitter for generic text. split documents recursively by different characters - starting with "\n\n", then "\n", then " "
        chunk_size=chunk_size,        # max size (in terms of number of characters) of the final documents
        chunk_overlap=chunk_overlap,  # how much overlap there should be between chunks
        add_start_index=True
    )
    chunks = text_splitter.split_documents(docs)
    logger.info(f"Split {len(docs)} documents into {len(chunks)} chunks.")
    return chunks


In [ ]:
chunks = get_chunks(docs)
chunks

[INFO] - Split 3 documents into 6 chunks. 


[Document(metadata={'source': 'docs/temario.txt', 'start_index': 0}, page_content='Contenidos:\nUnidad 1: Introducción a Transformers\nIntroducción al concepto de Gen AI, LLMs y Transformers. Historia. Arquitectura. Mecanismo de Atención. Embeddings y Positional Encoding. Aplicaciones en la industria.\n\nUnidad 2: Algoritmos de Embedding y Positional Encoding\nAlgoritmos de Embedding y Positional Encoding. Transformer basando en N-grama\n\nUnidad 3: Fine Tuning\nReinforcement Learning. RLHF y sus security issues. Fine tuning. Pipeline productivo. \n\nUnidad 4: Responsible AI\nConsideraciones éticas en AI: biases en training data, fairness, impacto social, detección de contenido generado de forma artificial. Narrow AI vs. AGI. AGI como agente. Foundation models. Emergent capabilities. Security vulnerabilities. Interpretability. Alignment.\n\nUnidad 5: Retrieval Augmented Generation (RAG)\nIntroducción Retrieval Augmented Generation. Bases de datos vectoriales: Chroma DB y Pinecone, uso 

 ### Note: Start_index metadata 📝
 When you need to reassemble the chunks into the original document format, start_index helps in placing each chunk at the correct position.

 Determine where in the original document each chunk belongs.

 # Embeddings 🤔

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings( #  embedding=OpenAIEmbeddings() rate limit
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'cuda'} #TODO CHANGE IF NOT USING GPU
)


/tmp/ipython-input-615253542.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings( #  embedding=OpenAIEmbeddings() rate limit
[INFO] - NumExpr defaulting to 2 threads. 
[DEBUG] - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client. 
[DEBUG] - Creating converter from 7 to 5 
[DEBUG] - Creating converter from 5 to 7 
[DEBUG] - Creating converter from 7 to 5 
[DEBUG] - Creating converter from 5 to 7 
DEBUG:2025-09-12 21:31:20,142:jax._src.path:41: etils.epath found. Using etils.epath for file I/O.
[DEBUG] - etils.epath found. Using etils.epath for fi

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[DEBUG] - Attempting to release lock 133658409676448 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/952a9b81c0bfd99800fabf352f69c7ccd46c5e43.lock 
[DEBUG] - Lock 133658409676448 released on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/952a9b81c0bfd99800fabf352f69c7ccd46c5e43.lock 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0 
[DEBUG] - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json HTTP/1.1" 200 0 
[DEBUG] - Attempting to acquire lock 133658175164576 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/fd1b291129c607e5d49799f87cb219b27f98acdf.lock 
[DEBUG] - Lock 133658175164576 acquired on /root/.cache/huggingface/hub/.locks/models--sentence-transform

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

[DEBUG] - Attempting to release lock 133658175164576 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/fd1b291129c607e5d49799f87cb219b27f98acdf.lock 
[DEBUG] - Lock 133658175164576 released on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/fd1b291129c607e5d49799f87cb219b27f98acdf.lock 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0 
[DEBUG] - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json HTTP/1.1" 200 0 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/1.1" 307 0 
[DEBUG] - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/README.md HTTP/1.1" 

...

[DEBUG] - Attempting to release lock 133658172890992 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/cb202bfe2e3c98645018a6d12f182a434c9d3e02.lock 
[DEBUG] - Lock 133658172890992 released on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/cb202bfe2e3c98645018a6d12f182a434c9d3e02.lock 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/added_tokens.json HTTP/1.1" 404 0 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/special_tokens_map.json HTTP/1.1" 307 0 
[DEBUG] - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/special_tokens_map.json HTTP/1.1" 200 0 
[DEBUG] - Attempting to acquire lock 133658172966000 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[DEBUG] - Attempting to release lock 133658172966000 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock 
[DEBUG] - Lock 133658172966000 released on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/e7b0375001f109a6b8873d756ad4f7bbb15fbaa5.lock 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/chat_template.jinja HTTP/1.1" 404 0 
[DEBUG] - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/1_Pooling/config.json HTTP/1.1" 307 0 
[DEBUG] - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/1_Pooling%2Fconfig.json HTTP/1.1" 200 0 
[DEBUG] - Attempting to acquire lock 133658175108880 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/d1514c3162bbe87b343f565fadc62e6c06f04f03.

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[DEBUG] - Attempting to release lock 133658175108880 on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/d1514c3162bbe87b343f565fadc62e6c06f04f03.lock 
[DEBUG] - Lock 133658175108880 released on /root/.cache/huggingface/hub/.locks/models--sentence-transformers--all-MiniLM-L6-v2/d1514c3162bbe87b343f565fadc62e6c06f04f03.lock 
[DEBUG] - https://huggingface.co:443 "GET /api/models/sentence-transformers/all-MiniLM-L6-v2 HTTP/1.1" 200 6861 


In [ ]:
vector = embeddings.embed_query("Hola como estas?")
embedding_size = len(vector)  # HF 384 ; OPENAI 1536
embedding_size

384

In [ ]:
vector

[-0.02919866144657135,
 0.09441244602203369,
 0.014229342341423035,
 0.021216703578829765,
 -0.08672457188367844,
 -0.002337127225473523,
 0.1574287861585617,
 -0.0299808531999588,
 0.0045777130872011185,
 0.016843819990754128,
 0.06859833002090454,
 0.0009345776052214205,
 -0.08560692518949509,
 0.002078711986541748,
 0.0845395177602768,
 0.026040565222501755,
 0.008749643340706825,
 -0.011601480655372143,
 0.006300992798060179,
 -0.00010649250907590613,
 0.054141316562891006,
 -0.03913291543722153,
 -0.08109074831008911,
 0.09663926064968109,
 -0.06758995354175568,
 -0.012158727273344994,
 -0.006165631581097841,
 0.04478027671575546,
 -0.02644125558435917,
 -0.07263953983783722,
 -0.03826495632529259,
 0.05620431900024414,
 0.01582055352628231,
 -0.02151469700038433,
 0.018006008118391037,
 -0.03673182800412178,
 0.06768332421779633,
 -0.11357313394546509,
 -0.01357504166662693,
 0.056269269436597824,
 -0.09015481919050217,
 -0.04250279441475868,
 -0.0032461422961205244,
 -0.01003888

In [ ]:
from langchain_openai import OpenAIEmbeddings  # 👈 updated import

# Use OpenAI embeddings instead of HuggingFace
openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # or "text-embedding-3-large"
)


In [ ]:
openai_vector = openai_embeddings.embed_query("Hola como estas?")
openai_embedding_size = len(openai_vector)  # HF 384 ; OPENAI 1536
openai_embedding_size

[DEBUG] - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-572ec77a-e16a-49a8-a8bc-1e96a2d2db18', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7814bd37f380>, 'json_data': {'input': [[69112, 8112, 48591, 30]], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}} 
[DEBUG] - Sending HTTP Request: POST https://api.openai.com/v1/embeddings 
[DEBUG] - close.started 
[DEBUG] - close.complete 
[DEBUG] - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None 
[DEBUG] - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7814bcab0140> 
[DEBUG] - start_tls.started ssl_context=<ssl.SSLContext object at 0x7814bcefc9d0> server_hostname='api.openai.com' timeout=None 
[DEBUG] - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7814bca56f60> 
[DEBUG] - send_request_headers.started request=<Request [b

1536

Para la próxima clase:

❓ Por que es mejor/peor la longitud del embedding vector?

❓ Qué algoritmo de embedding usa HF embeddings?

# Vector Store 🗃️

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

def get_vector_store(index_name, embeddings, embedding_size=384):
  """ Creates vector store from Pinecone for storing and managing embeddings.

    :param str index_name: The name of the index to create or retrieve from Pinecone.
    :param str embeddings: The embedding function to be used to generate embeddings
    :param int embedding_size: The size (dimension) of the embeddings. Defaults to 384 (e.g., for sentence-transformers/all-MiniLM-L6-v2).

    :return: PineconeVectorStore: An object representing the vector store in Pinecone for managing embeddings.

    :raise: ValueError: If the index creation fails due to invalid parameters or connection issues.
  """

  pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])  # Pinecone is initialized using an API key stored in the environment variable

  if INDEX_NAME not in pc.list_indexes().names():        # Check whether an index with the given index_name already exists
      pc.create_index(
          name=INDEX_NAME,          # Name of the index
          dimension=embedding_size, # Size of the vectors (embeddings)
          metric="cosine",          # Distance metric used to compare vectors
          spec=ServerlessSpec(      # Determines the infrastructure used
              cloud='aws',          # Specifies that the Pinecone index is hosted on AWS
              region='us-east-1'    # Specifies the region of the cloud provider
          )
      )

  vectorstore = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings) # initializes a PineconeVectorStore object using the index_name and the provided embeddings model or function

  return vectorstore

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


❗❗❗❗ VAMOS A PINECONE -> https://app.pinecone.io/

In [ ]:
INDEX_NAME = "langchain-demo-index-dl-20252q"
vectorstore = get_vector_store(INDEX_NAME, embeddings, embedding_size)

In [ ]:
vectorstore.add_documents(chunks)

['3eceadb3-7968-4e32-afe0-b21f881449a5',
 'bd3e9091-8044-45f0-be9b-31cbf2d67c11',
 'ff78f064-2c8f-409d-a0a8-37cee8fe6795',
 'f1ca081b-6a87-446d-bc9a-79d1e284d6ab',
 'bc2a5bbf-4160-4a0b-b2ab-0653cd248346',
 '5a168ba5-292f-4a57-989c-340046bcf540']

### Similarity Search 📝


- similarity: It retrieves the documents that are most similar to the query based on their embeddings, cos similarity

- MMR: Maximal Marginal Relevance balances the relevance of documents with the diversity of the results. It ensures that the returned documents are not only similar to the query but also diverse from each other

- Similarity Score Threshold: only those with a score above the threshold are included in the results.



In [ ]:
query = "quienes son los docentes?"
vectorstore.search(
    query=query,              # Return docs most similar to query using specified search type.
    search_type="similarity_score_threshold", # can be “similarity”, “mmr”, or “similarity_score_threshold”.
    k=4                       # return top k,
)

[Document(id='5a168ba5-292f-4a57-989c-340046bcf540', metadata={'source': 'docs/alumnos.txt', 'start_index': 0.0}, page_content='Las profesoras son la Ing. Eugenia Piñeiro y la Ing. Marina Fuster\nLos alumnos son estudiantes de la carrera ingeniería informática del ITBA'),
 Document(id='ff78f064-2c8f-409d-a0a8-37cee8fe6795', metadata={'source': 'docs/materia.txt', 'start_index': 0.0}, page_content='Denominación de la materia: Temas Avanzados de Deep Learning\n\n\n\nDocente responsable de la materia:  Eugenia Piñeiro\n\nEquipo docente: Marina Fuster y Eugenia Piñeiro\n\nCarga horaria total: 24 horas\n\nPresentación de la materia:\nEsta materia se enfoca en dos aspectos cruciales de la temática modelos de lenguaje. Por un lado, busca profundizar en el aspecto técnico de los mismos, construyendo sobre los pasos anteriores que llevan hasta su reciente desarrollo. \nPor otro lado, se tiene como objetivo entender el estado del arte y los desafíos teórico-prácticos que se encuentran abiertos e

There's no LLM processing yet!